In [ ]:
# https://github.com/Nixtla/neuralforecast/blob/main/nbs/models.deepar.ipynb
# https://github.com/Nixtla/neuralforecast/blob/main/nbs/losses.pytorch.ipynb

# DeepAR

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import Distribution
from torch.distributions import NegativeBinomial, Normal, Poisson
from torch.distributions import constraints

In [3]:
def level_to_outputs(level):
    qs = sum([[50-l/2, 50+l/2] for l in level], [])
    output_names = sum([[f'-lo-{l}', f'-hi-{l}'] for l in level], [])

    # Sort in increasing order
    sort_idx = np.argsort(qs)
    quantiles = np.array(qs)[sort_idx]
    output_names = list(np.array(output_names)[sort_idx])
    
    # Add median by default
    quantiles = np.concatenate([np.array([50]), quantiles])
    quantiles = torch.Tensor(quantiles) / 100
    output_names.insert(0, '-median')

    return quantiles, output_names

In [ ]:
def normal_scale_decouple(output, loc=None, scale=None, eps: float = 0.2):
    mean, std = output
    std = F.softplus(std)
    if (loc is not None) and (scale is not None):
        mean = (mean * scale) + loc
        std = (std + eps) * scale
    return mean, std


def nbinomial_scale_decouple(output, loc=None, scale=None):
    mu, alpha = output
    mu = F.softplus(mu) + 1e-08
    alpha = F.softplus(alpha) + 1e-08
    if (loc is not None) and (scale is not None):
        mu = mu * scale + loc
        alpha = alpha / (scale + 1.)

    total_count = 1.0 / alpha
    probs = (mu * alpha / (1.0 + mu * alpha)) + 1e-8
    return total_count, probs


def poisson_scale_decouple(output, loc=None, scale=None):
    eps = 1e-10
    rate, _ = output
    if (loc is not None) and (scale is not None):
        rate = (rate * scale) + loc
    rate = F.softplus(rate) + eps
    return (rate, )
    

In [8]:
# DistributionLoss

class DistributionLoss(nn.Module):
    def __init__(
        self,
        distribution: str,
        level=[80, 90],  # Confidence levels of prediction intervals
        num_samples=1000,
        return_params=False,
        **distribution_kwargs,
    ):
        super().__init__()
        qs, output_names = level_to_outputs(level)
        qs = torch.Tensor(qs)
        self.quantiles = torch.nn.Parameter(qs, requires_grad=False)
        self.output_names = output_names

        available_distributions = dict(
            Normal=Normal,
            NegativeBinomial=NegativeBinomial,
            Poisson=Poisson
        )
        scale_decouples = dict(
            Normal=normal_scale_decouple,
            NegativeBinomial=nbinomial_scale_decouple,
            Poisson=poisson_scale_decouple,
        )
        param_names = dict(
            Normal=["-loc", "-scale"],
            NegativeBinomial=["-total_count", "-logits"], 
            Poisson=["-loc"],
        )
        
        assert distribution in available_distributions
        self.distribution = distribution
        self._base_distribution = available_distributions[distribution]
        self.scale_decouple = scale_decouples[distribution]
        self.param_names = param_names[distribution]
        self.outputsize_multiplier = len(self.param_names)
        self.num_samples = num_samples
        self.return_params = return_params
        if self.return_params:
            self.output_names = self.output_names + self.param_names
        self.distribution_kwargs = distribution_kwargs
        
    
    def _domain_map(self, input: torch.Tensor):
        """
        Maps output of neural network to domain of distribution loss
        """
        output = torch.tensor_split(input, self.outputsize_multiplier, dim=2)
        return output
        
    
    def get_distribution(self, distr_args, **distribution_kwargs) -> Distribution:
        distr = self._base_distribution(*distr_args, **distribution_kwargs)
        self.distr_mean = distr.mean
        
        if self.distribution in ('Poisson', 'NegativeBinomial'):
              distr.support = constraints.nonnegative
        return distr
    
    def sample(self, distr_args: torch.Tensor, num_samples=None):
        if num_samples is None:
            num_samples = self.num_samples
        
        # Instantiate Scale Decoupled Distribution
        distr = self.get_distribution(distr_args=distr_args, **self.distribution_kwargs)
        samples = distr.sample(sample_shape=(num_samples,))
        samples = samples.permute(1, 2, 3, 0)  # [samples, B, H, N] -> [B, H, N, samples]
        
        # Compute mean and quantiles
        sample_mean = torch.mean(samples, dim=-1, keepdim=True)
        quants = torch.quantile(samples, self.quantiles, dim=-1)
        quants = quants.permute(1, 2, 3, 0)  # [Q, B, H, N] -> [B, H, N, Q]

        return samples, sample_mean, quants
    
    def __call__(self, y: torch.Tensor, distr_args: torch.Tensor):
        # Instantiate Scale Decoupled Distribution
        distr = self.get_distribution(distr_args=distr_args, **self.distribution_kwargs)
        loss_values = -distr.log_prob(y)
        return loss_values.mean()



In [12]:
class DeepARDecoder(nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        hidden_size: int,
        hidden_layers: int,
    ):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hidden_size = hidden_size
        self.hidden_layers = hidden_layers

        if hidden_layers == 0:
            # Linear transformation only
            layers = [nn.Linear(in_features=in_features, out_features=out_features)]
        else:
            # Input layer
            layers = [nn.Linear(in_features=in_features, out_features=hidden_size), nn.ReLU()]
            # Hidden layers
            for _ in range(hidden_layers - 2):
                layers += [nn.Linear(in_features=hidden_size, out_features=hidden_size), nn.ReLU()]
            # Output layer
            layers += [nn.Linear(in_features=hidden_size, out_features=out_features)]

        self.decoder = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.decoder(x)


class DeepARNormal(nn.Module):
    def __init__(
        self,    
        out_seq_length: int,
        input_seq_length: int = 1,
        input_size: int = 1,
        encoder_n_layers: int = 2,
        encoder_hidden_size: int = 128,
        encoder_dropout: float = 0.1,
        decoder_hidden_layers: int = 0,
        decoder_hidden_size: int = 0,
        decoder_output_size: int = 2,
        trajectory_samples: int = 100,
    ):
        super().__init__()
        self.out_seq_length = out_seq_length
        self.inp_seq_length = input_seq_length
        self.input_size = input_size
        
        # LSTM encoder
        self.encoder_n_layers = encoder_n_layers
        self.encoder_hidden_size = encoder_hidden_size
        self.encoder_dropout = encoder_dropout
        self.encoder = nn.LSTM(
            input_size=input_size,
            hidden_size=self.encoder_hidden_size,
            num_layers=self.encoder_n_layers,
            dropout=self.encoder_dropout,
            batch_first=True
        )

        # Decoder
        self.n_samples = trajectory_samples
        self.decoder = DeepARDecoder(
            in_features=encoder_hidden_size,
            out_features=decoder_output_size,
            hidden_size=decoder_hidden_size,
            hidden_layers=decoder_hidden_layers,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        hidden_state, _ = self.encoder(x)
        output = self.decoder(hidden_state)
        return output

In [20]:
decoder = DeepARDecoder(in_features=5, out_features=3, hidden_size=20, hidden_layers=3)
input = torch.rand((32, 6, 5))
output = decoder(input)

In [23]:
deepar = DeepARNormal(out_seq_length=16)
input = torch.randn(32, 21, 1)
output = deepar(input)